In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

An **input function** is a function that returns the following two-element tuple:
+ "features" - A Python dictionary in which:
 + Each key is the name of a feature.
 + Each value is an array containing all of that feature's values.
+ "label" - An array containing the values of the label for every example.

In [3]:
# Input function
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels
features, labels = input_evaluation_set()
print(features)
print(labels)

{'SepalLength': array([6.4, 5. ]), 'SepalWidth': array([2.8, 2.3]), 'PetalLength': array([5.6, 3.3]), 'PetalWidth': array([2.2, 1. ])}
[2 1]


# The Getting Started Example

In [4]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path
train_path, test_path = maybe_download()
print('train_path:', train_path)
print('test_path', test_path)

8192/2194 [================================================================================================================]8192/2194 [================================================================================================================] - 0s 0us/step

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================]8192/573 [===============================================================================================================================================================================================================================================================================================

In [25]:
# Load the Iris dataset
def load_data(y_name='Species'):    
    train_path, test_path = maybe_download()
    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)
    
    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)
    
    return (train_x, train_y), (test_x, test_y)

In [26]:
# Input function for training
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset
# For test
#(train_x, train_y), (test_x, test_y) = load_data()
#dict(train_x)[CSV_COLUMN_NAMES[0]][:5]

In [27]:
# Input function for evealuation or prediction
def eval_input_fn(features, labels, batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, 'batch_size must not be None'
    dataset = dataset.batch(batch_size)
    
    return dataset

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

In [41]:
# The model

batch_size = 100
train_steps = 1000

def main(argv):
    (train_x, train_y), (test_x, test_y) = load_data()

    # features columns describe how to use the input.
    my_feature_columns = []
    for key in train_x.keys(): # key is column name
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Build 2 hidden layer DNN with 10, 10 units respectively
    classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,
                                           hidden_units=[10, 10],
                                           n_classes=3)
    # Train the Model
    classifier.train(input_fn=lambda:train_input_fn(train_x, train_y, batch_size),
                    steps=train_steps)
    # Evaluate the Model
    eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))

    print('\nTest set accuracy:{accuracy:0.3f}\n'.format(**eval_result))
    
    # Generate predictions from the model
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(input_fn=lambda:eval_input_fn(predict_x, labels=None, batch_size=batch_size))
    
    for pred_dict, expec in zip(predictions, expected):
        print('Pred_dict:', pred_dict)
        print('expec', expec)
        template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

        class_id = pred_dict['class_ids'][0] # predict class
        probability = pred_dict['probabilities'][class_id] # the probability of predict class

        print(template.format(SPECIES[class_id], 100 * probability, expec))

In [42]:
tf.app.run(main)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Slash\\AppData\\Local\\Temp\\tmp58th21lc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F4D8ED3048>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Slash\AppData\Local\Temp\tmp58th21lc\model.ckpt.
INFO:tensorflow:loss = 204.07692, step = 1
INFO:tensorflow:global_step/sec: 191.904
INFO:tensorflow:loss = 21.336067, step = 101 (0.524 sec)
INFO:tensorflow:global_step/sec: 211.08
INFO:tensorflow:loss = 10.357854, step = 201 (0.473 s

SystemExit: 

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
